In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Definindo as horas do dia
horas = np.arange(0, 24)

# Simulação da curva de geração solar com uma distribuição aproximadamente normal
media = 12
desvio_padrao = 2.5  # Ajuste fino para adequar a curva ao intervalo de 6h às 18h
curva_normal = np.exp(-0.5 * ((horas - media) / desvio_padrao) ** 2)

# Normalizando a curva para que o pico seja 85% (0.85) da potência instalada
curva_normal = curva_normal / np.max(curva_normal) * 0.85

# Zerar a geração fora do intervalo de 6h às 18h
curva_normal[horas < 6] = 0
curva_normal[horas > 18] = 0

# Definindo o valor de pico da potência instalada (em kW)
potencia_pico = 10  # Ajuste esse valor para o pico desejado

# Calculando a potência gerada em cada hora do dia
potencia_gerada = potencia_pico * curva_normal

# Criando a tabela com pandas
tabela_geracao = pd.DataFrame({
    'Hora': horas,
    'Percentual (%)': curva_normal * 100,
    'Potência Gerada (kW)': potencia_gerada
})

# Exibindo a tabela
print(tabela_geracao)

# Plotando a curva de geração de energia solar
plt.plot(horas, potencia_gerada, label="Potência Gerada (kW)")
plt.xlabel("Hora do Dia")
plt.ylabel("Potência (kW)")
plt.title("Curva de Geração de Energia Solar (6h às 18h, Pico a 85%)")
plt.grid(True)
plt.legend()
plt.show()

# Clientes entrantes de GD

In [64]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

entrantes = pd.read_csv('entrantesGD.txt', sep=';', encoding='latin-1')


entrantes = entrantes.dropna(subset=['data_inicio_injecao'])

entrantes = entrantes[['num_uc_geradora', 'nomelcd', 'tipos_fonte_geracao',  'pot_instalada_solar', 'data_inicio_injecao']]

entrantes.rename(columns={
    'num_uc_geradora': 'UC',
    'nomelcd': 'Localidade',
    'tipos_fonte_geracao': 'Fonte',
    'pot_instalada_solar': 'PotenciaInstalada',
    'data_inicio_injecao': 'DataInicioInjecao'
}, inplace=True)

entrantes['PotenciaInstalada'] = entrantes['PotenciaInstalada'].str.replace(',', '.')

entrantes = entrantes.astype({
    'UC': 'int',
    'Localidade': 'object',
    'Fonte': 'object',
    'PotenciaInstalada': 'float64',
    'DataInicioInjecao': 'object'
})

entrantes = entrantes.dropna(subset='PotenciaInstalada')
entrantes = entrantes.drop_duplicates()

#############################################################

uc_trafo = pd.read_csv('UC_Trafo.txt', sep=';', encoding='latin-1',dtype=object)
trafo_alim = pd.read_csv('Trafo_ID.txt', sep=';', encoding='latin-1',dtype=object)

uc_alim = pd.merge(uc_trafo, trafo_alim, on='Instalacao')
uc_alim = uc_alim[['Conta', 'Circuit_Id']]
uc_alim['Conta'] = uc_alim['Conta'].str.slice(2).astype(int)
uc_alim

uc_alim.rename(columns={
    'Conta': 'UC',
    'Circuit_Id': 'ALIMENTADOR'
}, inplace=True)

df = pd.merge(entrantes, uc_alim, on='UC')

#############################################################

sub = pd.read_excel('subestacoes.xlsx')
sub['TRAFO'] = sub['SUBESTACAO'] + "_DJ_" + sub['DISJUNTOR']
df2 = pd.merge(df, sub, on='ALIMENTADOR')
df2['PotenciaInstalada_MW'] = df2['PotenciaInstalada']/1000
df2 = df2[['UC', 'PotenciaInstalada_MW', 'DataInicioInjecao', 'TRAFO']]
df2['DataInicioInjecao'] = pd.to_datetime(df2['DataInicioInjecao'], format='%d/%m/%Y %H:%M:%S')

df2


,UC,PotenciaInstalada_MW,DataInicioInjecao,TRAFO
0,2019148,0.010560,2020-09-26 15:32:33,CTR_DJ_12B1
1,2146954,0.040800,2021-03-16 09:07:00,CTR_DJ_12B1
2,1497237,0.014625,2024-11-19 11:59:06,CTR_DJ_12B1
3,2282258,0.075000,2024-09-30 16:46:00,CTR_DJ_12B1
4,2282258,0.074800,2022-10-14 16:21:00,CTR_DJ_12B1
...,...,...,...,...
32605,332989,0.050000,2021-11-24 07:54:00,MAA_DJ_12B1
32606,1448169,0.150000,2020-07-07 09:13:00,MRO_DJ_12B1
32607,813176,0.005000,2022-11-09 17:00:00,MRO_DJ_12B1
32608,1739724,0.075000,2023-09-25 11:49:00,MRO_DJ_12B1


In [65]:
# Função para calcular a potência acumulada para um grupo específico (um TRAFO)
def calcular_potencia_acumulada_por_trafo(grupo):
    # Criar DataFrame de eventos de adição (injeção de potência) para o grupo
    eventos_adicao = grupo[['DataInicioInjecao', 'PotenciaInstalada_MW']].rename(columns={'DataInicioInjecao': 'Tempo', 'PotenciaInstalada_MW': 'Potencia'})
    eventos_adicao['Tipo'] = 'adicao'

    # Criar DataFrame com as potências de hora em hora
    start_time = grupo['DataInicioInjecao'].min().floor('H')
    end_time = pd.Timestamp.now().floor('H')  # Define o tempo final para o cálculo
    time_range = pd.date_range(start=start_time, end=end_time, freq='H')
    potencia_instalada = pd.DataFrame({'Tempo': time_range})

    # Inicializar a potência acumulada
    potencia_instalada['PotenciaAcumulada'] = 0

    # Ordenar eventos por tempo
    eventos_adicao = eventos_adicao.sort_values(by='Tempo').reset_index(drop=True)

    # Calcular a potência acumulada
    potencia = 0
    event_index = 0
    num_events = len(eventos_adicao)

    for i in range(len(potencia_instalada)):
        current_time = potencia_instalada.loc[i, 'Tempo']

        # Atualiza a potência acumulada com base nos eventos de adição
        while event_index < num_events and eventos_adicao.loc[event_index, 'Tempo'] <= current_time:
            potencia += eventos_adicao.loc[event_index, 'Potencia']
            event_index += 1

        potencia_instalada.loc[i, 'PotenciaAcumulada'] = potencia

    return potencia_instalada

# Aplicar a função para cada grupo (TRAFO) separadamente
resultados = []

# Iterar sobre cada TRAFO
for trafo, grupo in df2.groupby('TRAFO'):
    # Calcular a potência acumulada para o grupo atual (TRAFO)
    potencia_acumulada_por_trafo = calcular_potencia_acumulada_por_trafo(grupo)
    potencia_acumulada_por_trafo['TRAFO'] = trafo  # Adicionar coluna TRAFO
    resultados.append(potencia_acumulada_por_trafo)

potencia_instalada_final = pd.concat(resultados).reset_index(drop=True)

C:\Users\rcvs3\AppData\Local\Temp\ipykernel_24936\1302196479.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.01125' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  potencia_instalada.loc[i, 'PotenciaAcumulada'] = potencia
C:\Users\rcvs3\AppData\Local\Temp\ipykernel_24936\1302196479.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.02' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  potencia_instalada.loc[i, 'PotenciaAcumulada'] = potencia
C:\Users\rcvs3\AppData\Local\Temp\ipykernel_24936\1302196479.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.006' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  potencia_instalada.loc[i, 'PotenciaAc

In [66]:
potencia_instalada_final

,Tempo,PotenciaAcumulada,TRAFO
0,2019-02-12 15:00:00,0.00000,ABR_DJ_12B1
1,2019-02-12 16:00:00,0.01125,ABR_DJ_12B1
2,2019-02-12 17:00:00,0.01125,ABR_DJ_12B1
3,2019-02-12 18:00:00,0.01125,ABR_DJ_12B1
4,2019-02-12 19:00:00,0.01125,ABR_DJ_12B1
...,...,...,...
4419442,2025-02-03 09:00:00,2.63609,URN_DJ_12B1
4419443,2025-02-03 10:00:00,2.63609,URN_DJ_12B1
4419444,2025-02-03 11:00:00,2.63609,URN_DJ_12B1
4419445,2025-02-03 12:00:00,2.63609,URN_DJ_12B1


In [68]:
potencia_instalada_final.to_csv('EntrantesGD.csv', sep=';',index=False)

# ANTIGO

In [ ]:
clientes = pd.read_csv('clientesEPB.csv', sep=';', encoding='latin-1')
# d3 = pd.concat([d1, d2])

entrantes = entrantes.dropna(subset=['data_inicio_injecao'])

entrantes = entrantes[['num_uc_geradora', 'nomelcd', 'tipos_fonte_geracao',  'pot_instalada_solar', 'data_inicio_injecao']]

entrantes.rename(columns={
    'num_uc_geradora': 'UC',
    'nomelcd': 'Localidade',
    'tipos_fonte_geracao': 'Fonte',
    'pot_instalada_solar': 'PotenciaInstalada',
    'data_inicio_injecao': 'DataInicioInjecao'
}, inplace=True)

entrantes['PotenciaInstalada'] = entrantes['PotenciaInstalada'].str.replace(',', '.')

entrantes = entrantes.astype({
    'UC': 'int',
    'Localidade': 'object',
    'Fonte': 'object',
    'PotenciaInstalada': 'float64',
    'DataInicioInjecao': 'object'
})


clientes = clientes[['UC', 'ALIMENTADOR', 'SUBESTACAO', 'DATA_ENCERRAMENTO_DO_CONTRATO',
       'LIVRO', 'NOME_LOCALIDADE', 'LAT_SIATE',
       'LONG_SIATE', 'LAT_SILCO', 'LONG_SILCO']]


clientes['UC'] = pd.to_numeric(clientes['UC'], errors='coerce')
clientes = clientes.dropna(subset=['UC'])
clientes = clientes.astype({
    'UC': 'int'
})

geradores = pd.merge(entrantes, clientes, on='UC',how='left')

import pandas as pd

def trocar_colunas(df):
    for index, row in df.iterrows():        
        alimentador = row['ALIMENTADOR']
        subestacao = row['SUBESTACAO']
        
        # Verificar se o alimentador tem o formato de subestação e vice-versa
        if isinstance(alimentador, str) and isinstance(subestacao, str):
            if len(alimentador) == 3: #and alimentador.isupper() and 'L' in subestacao:
                # Troca os valores se estiverem trocados
                df.at[index, 'ALIMENTADOR'] = subestacao
                df.at[index, 'SUBESTACAO'] = alimentador

    return df

geradores['SUBESTACAO'] = geradores['SUBESTACAO'].str.replace(' ', '').str[:3]
geradores['ALIMENTADOR'] = geradores['ALIMENTADOR'].str.replace(' ', '').str[:3]

geradores = trocar_colunas(geradores)

geradores['CIRCUITO'] = geradores['SUBESTACAO']+'-'+geradores['ALIMENTADOR']

subestacoes = pd.read_excel('subestacoes.xlsx', usecols='A:G')
subestacoes['Equipamento Medição'] = subestacoes['SUBESTACAO']+'_DJ_'+subestacoes['DISJUNTOR']
subestacoes['TRAFO'] = subestacoes['SUBESTACAO']+"-"+subestacoes['TRAFO']
subestacoes = subestacoes[['TRAFO', 'Equipamento Medição', 'ALIMENTADOR', 'POTENCIA (MVA)']]

geradores_e_ses = pd.merge(geradores, subestacoes, left_on='CIRCUITO', right_on='ALIMENTADOR')
geradores_e_ses = geradores_e_ses.drop_duplicates()

geradores_e_ses['PotenciaInstalada_MW'] = geradores_e_ses['PotenciaInstalada']/1000

geradores_e_ses = geradores_e_ses[['Localidade', 'Fonte', 'DataInicioInjecao', 'CIRCUITO', 'TRAFO', 'Equipamento Medição', 'POTENCIA (MVA)', 'PotenciaInstalada_MW']]


df = geradores_e_ses

df['DataInicioInjecao'] = pd.to_datetime(df['DataInicioInjecao'], format='%d/%m/%Y %H:%M:%S')

# Função para calcular a potência acumulada para um grupo específico (um TRAFO)
def calcular_potencia_acumulada_por_trafo(grupo):
    # Criar DataFrame de eventos de adição (injeção de potência) para o grupo
    eventos_adicao = grupo[['DataInicioInjecao', 'PotenciaInstalada_MW']].rename(columns={'DataInicioInjecao': 'Tempo', 'PotenciaInstalada_MW': 'Potencia'})
    eventos_adicao['Tipo'] = 'adicao'

    # Criar DataFrame com as potências de hora em hora
    start_time = grupo['DataInicioInjecao'].min().floor('H')
    end_time = pd.Timestamp.now().floor('H')  # Define o tempo final para o cálculo
    time_range = pd.date_range(start=start_time, end=end_time, freq='H')
    potencia_instalada = pd.DataFrame({'Tempo': time_range})

    # Inicializar a potência acumulada
    potencia_instalada['PotenciaAcumulada'] = 0

    # Ordenar eventos por tempo
    eventos_adicao = eventos_adicao.sort_values(by='Tempo').reset_index(drop=True)

    # Calcular a potência acumulada
    potencia = 0
    event_index = 0
    num_events = len(eventos_adicao)

    for i in range(len(potencia_instalada)):
        current_time = potencia_instalada.loc[i, 'Tempo']

        # Atualiza a potência acumulada com base nos eventos de adição
        while event_index < num_events and eventos_adicao.loc[event_index, 'Tempo'] <= current_time:
            potencia += eventos_adicao.loc[event_index, 'Potencia']
            event_index += 1

        potencia_instalada.loc[i, 'PotenciaAcumulada'] = potencia

    return potencia_instalada

# Aplicar a função para cada grupo (TRAFO) separadamente
resultados = []

# Iterar sobre cada TRAFO
for trafo, grupo in df.groupby('TRAFO'):
    # Calcular a potência acumulada para o grupo atual (TRAFO)
    potencia_acumulada_por_trafo = calcular_potencia_acumulada_por_trafo(grupo)
    potencia_acumulada_por_trafo['TRAFO'] = trafo  # Adicionar coluna TRAFO
    resultados.append(potencia_acumulada_por_trafo)

# Concatenar os resultados de todos os trafos
potencia_instalada_final = pd.concat(resultados).reset_index(drop=True)